### Notebook Description

This notebook walks through the implementation in the `integrateODEFunction` script. The current implementation uses the `odeint` model to integrate the function defined by the ODE function.

#### Initialization

In [ ]:
import os, sys
NotebookDirectory = os.path.split(os.getcwd())[0]
if NotebookDirectory not in sys.path:
    sys.path.append(NotebookDirectory)
    
import toolbox 

SciPyModel = toolbox.sbml.importSBMLFile(toolbox.createSciPyModel())
SciPyModel = toolbox.simulation.writeODEFunction(SciPyModel)

### Simulating SciPyModel

#### Write Temporary File

In order to use the file it needs to be written to a temporary file. To keep the name unique we use the model name with the current date and time in the name of the temporary file.

In [ ]:
import datetime
TempName = SciPyModel.MetaData.Name+datetime.datetime.now().strftime("%I%M%p%B%d%Y")+'.py'
open(TempName, 'w+').write(SciPyModel.ToolboxFunctions.DerivativeFunction)

Then we can import the function from the file using a temporary module declaration.

In [ ]:
TempModule = __import__(TempName[:-3])

#### Using Internal Declarations

The `SciPyModel` object contains vectors of each state variable and parameter. This makes simulation using `SciPy` straightforward.

In [1]:
SciPyModel.SimulationData.TimeStart = 0
SciPyModel.SimulationData.TimeEnd = 100
SciPyModel.SimulationData.DataPoints = 100

NameError: name 'SciPyModel' is not defined

#### Deterministically Simulating SciPyModel

Using the `odeint` function we simply need to call our information from `SciPyModel`.

In [2]:
from scipy.integrate import odeint
import numpy

SciPyModel.SimulationData.Deterministic.Data = (odeint(
    TempModule.ode_fun,
    SciPyModel.Species.Value,
    numpy.linspace(SciPyModel.SimulationData.TimeStart,
                   SciPyModel.SimulationData.TimeEnd,
                   SciPyModel.SimulationData.DataPoints),
    args=(SciPyModel.Parameters.Value, )))

NameError: name 'TempModule' is not defined

#### Final Notes

Once the desired simulations with the model are completed, the simulation data can be exported and the temporary files can be deleted.

In [3]:
import os
try:
    os.remove(TempName)
except OSError:
    print 'ERROR: Temporary file has already been deleted.'
else:
    print 'ERROR: Unknown error. Unable to remove '+TempName

NameError: name 'TempName' is not defined

#### Plotting Simulation

Plotting is made possible by using the `matplotlib` module. Several plotting functions have been written to make code easier to read and keep focus on the model development process.